# Some visualization in habitat

References:
+ [habitat-sim colab example](https://github.com/facebookresearch/habitat-sim/blob/main/examples/tutorials/colabs/ECCV_2020_Navigation.ipynb)
+ [habitat official tutorials](https://aihabitat.org/tutorial/2020/)
+ [habitat doc](https://aihabitat.org/docs/habitat-lab/)


In [1]:
import os

# Avoid too much logs
os.environ["MAGNUM_LOG"] = "quiet"
os.environ["GLOG_minloglevel"] = "2"
os.environ["HABITAT_SIM_LOG"] = "quiet"

import habitat_sim
from habitat_sim.utils import common as utils
from habitat_sim.utils import viz_utils as vut
from habitat_sim.utils.common import d3_40_colors_rgb
from habitat_baselines.utils.common import center_crop
import habitat
from habitat.utils.visualizations import maps 

import magnum as mn
import math
import random
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import json
import gzip

import numpy as np
def display_sample(rgb_obs, semantic_obs, depth_obs, idx=0, trajectory_id=0):
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    rgb_img.save("trajectory%d_%d_rgb.png"%(trajectory_id,idx))
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    semantic_img.save("trajectory%d_%d_semantic.png"%(trajectory_id,idx))
    
    depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
    depth_img.save("trajectory%d_%d_depth.png"%(trajectory_id,idx))

    arr = [rgb_img, semantic_img, depth_img]
    titles = ['rgb', 'semantic', 'depth']
    plt.figure(figsize=(12 ,8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i+1)
        ax.axis('off')
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show()
def display_map(topdown_map, key_points=None):
    plt.figure(figsize=(12, 8))
    ax = plt.subplot(1, 1, 1)
    ax.axis("off")
    plt.imshow(topdown_map)
    # plot points on map
    if key_points is not None:
        for point in key_points:
            plt.plot(point[0], point[1], marker="o", markersize=10, alpha=0.8)
    plt.show(block=False)

# Simulator config

In [ ]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_id = settings["scene"]
    
    # Note: all sensors must have the same resolution
    sensors = {
        "color_sensor": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "depth_sensor": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "semantic_sensor": {
            "sensor_type": habitat_sim.SensorType.SEMANTIC,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },  
    }
    
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.CameraSensorSpec() # SensorSpec
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]

            sensor_specs.append(sensor_spec)
            
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
    }
    
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])
test_scene = "/share/home/tj90055/hzt/IA-HWP/data/scene_datasets/mp3d/8WUmhLawc2A/8WUmhLawc2A.glb"
start_point = np.array([
                -6.2812700271606445,
                0.10065700113773346,
                -0.39065900444984436
            ])
start_rotation = np.array([
                0,
                -0.7306023269338372,
                0,
                0.6828032219306397
            ])
end_point = np.array([
                        -16.707500457763672,
                        0.10065700113773346,
                        -8.674070358276367
                    ])
sim_settings = {
    "width": 512,  # Spatial resolution of the observations    
    "height": 512,
    "scene": test_scene,  # Scene path
    "default_agent": 0,  
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": True,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
}
trajectory_id = 607
cfg = make_cfg(sim_settings)
sim = habitat_sim.Simulator(cfg)

In [ ]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])

# Set agent state
agent = sim.initialize_agent(sim_settings["default_agent"])
agent_state = habitat_sim.AgentState()
agent_state.position = start_point
agent_state.rotation = start_rotation
agent.set_state(agent_state)

# Get agent state
agent_state = agent.get_state()
print("agent_state: position", agent_state.position, "rotation", agent_state.rotation)

# Take random actions and display sensor data

In [ ]:
total_frames = 0
action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)

max_frames = 5

while total_frames < max_frames:
    action = random.choice(action_names)
    print("action", action)
    observations = sim.step(action)
    rgb = observations["color_sensor"]
    print(rgb.shape)
    semantic = observations["semantic_sensor"]
    depth = observations["depth_sensor"]
    
    display_sample(rgb, semantic, depth)
    
    total_frames += 1

## Set position and save image

In [ ]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])

# Set agent state
agent = sim.initialize_agent(sim_settings["default_agent"])
agent_state = habitat_sim.AgentState()
agent_state.position = start_point
agent_state.rotation = start_rotation
agent.set_state(agent_state)

# Get agent state
agent_state = agent.get_state()
print("agent_state: position", agent_state.position, "rotation", agent_state.rotation)

# Get observations
observations = sim.get_sensor_observations()
rgb = observations["color_sensor"]
print(rgb.shape)
semantic = observations["semantic_sensor"]
depth = observations["depth_sensor"]
display_sample(rgb, semantic, depth)

## Display top down map

In [ ]:
meters_per_pixel = 0.1
nav_points = start_point
top_down_map = maps.get_topdown_map(
    sim.pathfinder, height=nav_points[1], meters_per_pixel=meters_per_pixel
)
recolor_map = np.array(
    [[255, 255, 255], [128, 128, 128], [0, 0, 0]], dtype=np.uint8
)
top_down_map = recolor_map[top_down_map]
print("\nDisplay the map with key_point overlay:")
display_map(top_down_map, key_points=[nav_points])

## Display map and shortest path

In [ ]:

seed = 4  # @param {type:"integer"}
sim.pathfinder.seed(seed)
start_point = np.array([-6.2812700271606445,0.10065700113773346,-0.39065900444984436])
start_rotation = np.array([0,-0.7306023269338372,0,0.6828032219306397])
end_point = np.array([-16.707500457763672,0.10065700113773346,-8.674070358276367])
# fmt off
# @markdown 1. Sample valid points on the NavMesh for agent spawn location and pathfinding goal.
# fmt on
sample1 = start_point
sample2 = end_point

# @markdown 2. Use ShortestPath module to compute path between samples.
path = habitat_sim.ShortestPath()
path.requested_start = sample1
path.requested_end = sample2

found_path = sim.pathfinder.find_path(path)
geodesic_distance = path.geodesic_distance
path_points = path.points
# @markdown - Success, geodesic path length, and 3D points can be queried.
print("found_path : " + str(found_path))
print("geodesic_distance : " + str(geodesic_distance))
print("path_points : " + str(path_points))

# @markdown 3. Display trajectory (if found) on a topdown map of ground floor
if found_path:
    meters_per_pixel = 0.025
    scene_bb = sim.get_active_scene_graph().get_root_node().cumulative_bb
    height = scene_bb.y().min
    if display:
        top_down_map = maps.get_topdown_map(
            sim.pathfinder, height, meters_per_pixel=meters_per_pixel
        )
        recolor_map = np.array(
            [[255, 255, 255], [128, 128, 128], [0, 0, 0]], dtype=np.uint8
        )
        top_down_map = recolor_map[top_down_map]
        grid_dimensions = (top_down_map.shape[0], top_down_map.shape[1])
        # convert world trajectory points to maps module grid points
        trajectory = [
            maps.to_grid(
                path_point[2],
                path_point[0],
                grid_dimensions,
                pathfinder=sim.pathfinder,
            )
            for path_point in path_points
        ]
        grid_tangent = mn.Vector2(
            trajectory[1][1] - trajectory[0][1], trajectory[1][0] - trajectory[0][0]
        )
        path_initial_tangent = grid_tangent / grid_tangent.length()
        initial_angle = math.atan2(path_initial_tangent[0], path_initial_tangent[1])
        # draw the agent and trajectory on the map
        maps.draw_path(top_down_map, trajectory)
        maps.draw_agent(
            top_down_map, trajectory[0], initial_angle, agent_radius_px=8
        )
        print("\nDisplay the map with agent and path overlay:")
        display_map(top_down_map)

    # @markdown 4. (optional) Place agent and render images at trajectory points (if found).    
    display_path_agent_renders = True  # @param{type:"boolean"}
    final_view = True
    if display_path_agent_renders:
        print("Rendering observations at path points:")
        tangent = path_points[1] - path_points[0]
        agent_state = habitat_sim.AgentState()
        for ix, point in enumerate(path_points):
            if ix < len(path_points) - 1:
                tangent = path_points[ix + 1] - point
                agent_state.position = point
                tangent_orientation_matrix = mn.Matrix4.look_at(
                    point, point + tangent, np.array([0, 1.0, 0])
                )
                tangent_orientation_q = mn.Quaternion.from_matrix(
                    tangent_orientation_matrix.rotation()
                )
                agent_state.rotation = utils.quat_from_magnum(tangent_orientation_q)
                if ix==0:
                    agent_state.rotation = start_rotation
                agent.set_state(agent_state)
                
                observations = sim.get_sensor_observations()
                rgb = observations["color_sensor"]
                semantic = observations["semantic_sensor"]
                depth = observations["depth_sensor"]

                # if display:
                display_sample(rgb, semantic, depth, idx=ix)
    final_view = True
    if final_view:
        point = path_points[-1]
        agent_state.position = point
        agent_state.rotation = start_rotation
        agent.set_state(agent_state)
        observations = sim.get_sensor_observations()
        rgb = observations["color_sensor"]
        semantic = observations["semantic_sensor"]
        depth = observations["depth_sensor"]

        # if display:
        display_sample(rgb, semantic, depth, idx=len(path_points))
sim.close()

## Display map and shortest path with panoramas

In [66]:
try:
    sim.close()
except:
    pass

IMAGE_SIZE = 256
VIEW_NUM = 72
SPLIT_NUM = 90 / (360/VIEW_NUM)
OFFSET = IMAGE_SIZE*200/256/VIEW_NUM # 6
# OFFSET = 0
# test_scene = "/share/home/tj90055/hzt/IA-HWP/data/scene_datasets/mp3d/zsNo4HB9uLZ/zsNo4HB9uLZ.glb"
test_scene =  "/share/home/tj90055/hzt/IA-HWP/data/scene_datasets/mp3d/zsNo4HB9uLZ/zsNo4HB9uLZ.glb"
ROTATE_CENTER = True
SHORTEST_PATH = False
EPISODE_ID = 251 # 154
VAL_SPLIT = "val_unseen"
DATA_FOLDER = "/share/home/tj90055/hzt/IA-HWP/data/datasets/R2R_VLNCE_v1-2_preprocessed"

def get_camera_orientations(sectors=12):
    base_angle_deg = 360 / sectors
    base_angle_rad = math.pi / (sectors / 2)
    orient_dict = {}
    for k in range(0,sectors):
        orient_dict[str(base_angle_deg*k)] = [0.0, base_angle_rad*k, 0.0]
    return orient_dict
def make_cfg_pano(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_id = settings["scene"]
    sensors = {}
    orientations = get_camera_orientations(VIEW_NUM)
    for i, o in orientations.items():
        if settings["color_sensor"]:
            sensors["color_sensor_{}".format(i)] = {
                    "sensor_type": habitat_sim.SensorType.COLOR,
                    "resolution": [settings["height"], settings["width"]],
                    "position": [0.0, settings["sensor_height"], 0.0],
                    "orientation": o,
                    "hfov": 90,
                }
        if settings["depth_sensor"]:
            sensors["depth_sensor_{}".format(i)] = {
                    "sensor_type": habitat_sim.SensorType.DEPTH,
                    "resolution": [settings["height"], settings["width"]],
                    "position": [0.0, settings["sensor_height"], 0.0],
                    "orientation": o,
                    "hfov": 90,
                }
        if settings["semantic_sensor"]:
            sensors["semantic_sensor_{}".format(i)] = {
                    "sensor_type": habitat_sim.SensorType.SEMANTIC,
                    "resolution": [settings["height"], settings["width"]],
                    "position": [0.0, settings["sensor_height"], 0.0],
                    "orientation": o,
                    "hfov": 90,
                }
    
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        sensor_spec = habitat_sim.CameraSensorSpec() # SensorSpec
        sensor_spec.uuid = sensor_uuid
        sensor_spec.sensor_type = sensor_params["sensor_type"]
        sensor_spec.resolution = sensor_params["resolution"]
        sensor_spec.position = sensor_params["position"]
        sensor_spec.orientation = sensor_params["orientation"]
        sensor_spec.hfov = sensor_params["hfov"]
        # sensor_spec.VFOV = sensor_params["VFOV"]

        sensor_specs.append(sensor_spec)
            
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
    }
    
    rgb_list = ["color_sensor_{}".format(v) for v in orientations]
    return habitat_sim.Configuration(sim_cfg, [agent_cfg]), rgb_list

def stitch_images(images):
    """Not well"""
    # Create a Stitcher object
    stitcher = cv2.Stitcher_create(cv2.Stitcher_PANORAMA)
    
    # Perform the stitching process
    status, stitched_image = stitcher.stitch(images)
    
    if status == cv2.Stitcher_OK:
        print("Stitching completed successfully.")
        return stitched_image
    else:
        print(f"Error during stitching: {status}")
        return None
def get_pano(observations, need_depth=False, observation_size=(2048,512)):
    pano_rgb = []
    pano_depth = []
    
    rgb_frame = []
    depth_frame = []
    for i in range(VIEW_NUM-1,-1,-1):
        rgb_frame.append(center_crop(observations[RGB_LIST[i]][:,:,:3],  (int(IMAGE_SIZE/SPLIT_NUM - OFFSET), IMAGE_SIZE)))
        if need_depth:
            depth = (observations[DEPTH_LIST[i]].squeeze() * 255).astype(np.uint8)
            depth = np.stack([depth for _ in range(3)], axis=2)
            depth = center_crop(depth, (int(IMAGE_SIZE/SPLIT_NUM - OFFSET), IMAGE_SIZE))
            depth_frame.append(depth)
    pano_rgb = np.concatenate(rgb_frame, axis=1)
    if ROTATE_CENTER:
        pano_rgb = np.roll(pano_rgb, pano_rgb.shape[1]//2, axis=1)
    if need_depth:
        pano_depth = np.concatenate(depth_frame, axis=0)
        if ROTATE_CENTER:
            pano_depth = np.roll(pano_depth, pano_depth.shape[1]//2, axis=1)
    else:
        pano_depth = None
    return pano_rgb, pano_depth

def display_sample_pano(obs, idx=0, trajectory_id=0):
    rgb_pano, _ = get_pano(obs)

    # plt.figure(figsize=(16 ,4))
    plt.imshow(rgb_pano)
    plt.axis(False)
    plt.tight_layout()
    plt.savefig(f"{trajectory_id}_{idx}.png", dpi=300, bbox_inches="tight", pad_inches=0)
    plt.show()
    plt.close()

sim_settings = {
    "width": IMAGE_SIZE,  # Spatial resolution of the observations    
    "height": IMAGE_SIZE,
    "scene": test_scene,  # Scene path
    "default_agent": 0,  
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": False,  # Semantic sensor
    "depth_sensor": False,  # Depth sensor
    "seed": 1,
}

cfg, RGB_LIST = make_cfg_pano(sim_settings)
DEPTH_LIST = [v.replace("color", "depth") for v in RGB_LIST]
sim = habitat_sim.Simulator(cfg)

# total_frames = 0
# action_names = list(
#     cfg.agents[
#         sim_settings["default_agent"]
#     ].action_space.keys()
# )
# max_frames = 2
# while total_frames < max_frames:
#     action = random.choice(action_names)
#     print("action", action)
#     observations = sim.step(action)
    
#     display_sample_pano(observations)
    
#     total_frames += 1

In [ ]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])
sim.pathfinder.seed(sim_settings["seed"])

with gzip.open(os.path.join(DATA_FOLDER, VAL_SPLIT, VAL_SPLIT+"_gt.json.gz"), "r") as f:
    gt = json.load(f)
with gzip.open(os.path.join(DATA_FOLDER, VAL_SPLIT, VAL_SPLIT+".json.gz"), "r") as f:
    split_data = json.load(f)
traj = gt[str(EPISODE_ID)]
for v in split_data["episodes"]:
    if v["episode_id"] == EPISODE_ID:
        episode = v
        break

start_point = np.array(episode["start_position"])
start_rotation = np.array(episode["start_rotation"])
end_point = np.array(episode["goals"][0]["position"])
# Set agent state
agent = sim.initialize_agent(sim_settings["default_agent"])

if SHORTEST_PATH:
    sample1 = start_point
    sample2 = end_point
    
    path = habitat_sim.ShortestPath()
    path.requested_start = sample1
    path.requested_end = sample2
    
    found_path = sim.pathfinder.find_path(path)
    geodesic_distance = path.geodesic_distance
    path_points = path.points
else:
    found_path = True
    geodesic_distance = episode["info"]["geodesic_distance"]
    path_points = [np.array(v) for v in traj["locations"]]
    
print("found_path : " + str(found_path))
print("geodesic_distance : " + str(geodesic_distance))
print("path_points : " + str(path_points))
# @markdown 3. Display trajectory (if found) on a topdown map of ground floor
if found_path:
    meters_per_pixel = 0.025
    scene_bb = sim.get_active_scene_graph().get_root_node().cumulative_bb
    height = scene_bb.y().min
    top_down_map = maps.get_topdown_map(
        sim.pathfinder, height, meters_per_pixel=meters_per_pixel
    )
    recolor_map = np.array(
        [[255, 255, 255], [128, 128, 128], [0, 0, 0]], dtype=np.uint8
    )
    top_down_map = recolor_map[top_down_map]
    grid_dimensions = (top_down_map.shape[0], top_down_map.shape[1])
    # convert world trajectory points to maps module grid points
    trajectory = [
        maps.to_grid(
            path_point[2],
            path_point[0],
            grid_dimensions,
            pathfinder=sim.pathfinder,
        )
        for path_point in path_points
    ]
    grid_tangent = mn.Vector2(
        trajectory[1][1] - trajectory[0][1], trajectory[1][0] - trajectory[0][0]
    )
    path_initial_tangent = grid_tangent / grid_tangent.length()
    initial_angle = math.atan2(path_initial_tangent[0], path_initial_tangent[1])
    # draw the agent and trajectory on the map
    maps.draw_path(top_down_map, trajectory)
    maps.draw_agent(
        top_down_map, trajectory[0], initial_angle, agent_radius_px=8
    )
    print("\nDisplay the map with agent and path overlay:")
    display_map(top_down_map)

    # @markdown 4. (optional) Place agent and render images at trajectory points (if found).    
    display_path_agent_renders = True  # @param{type:"boolean"}
    final_view = True
    if display_path_agent_renders:
        print("Rendering observations at path points:")
        tangent = path_points[1] - path_points[0]
        agent_state = habitat_sim.AgentState()
        for ix, point in enumerate(path_points):
            if ix < len(path_points) - 1:
                tangent = path_points[ix + 1] - point
                agent_state.position = point
                tangent_orientation_matrix = mn.Matrix4.look_at(
                    point, point + tangent, np.array([0, 1.0, 0])
                )
                tangent_orientation_q = mn.Quaternion.from_matrix(
                    tangent_orientation_matrix.rotation()
                )
                agent_state.rotation = utils.quat_from_magnum(tangent_orientation_q)
                if ix==0:
                    agent_state.rotation = start_rotation
                agent.set_state(agent_state)
                
                observations = sim.get_sensor_observations()

                # if display:
                display_sample_pano(observations, idx=ix)
    final_view = True
    if final_view:
        point = path_points[-1]
        agent_state.position = point
        agent_state.rotation = utils.quat_from_magnum(tangent_orientation_q)
        agent.set_state(agent_state)
        observations = sim.get_sensor_observations()

        # if display:
        display_sample_pano(observations, idx=len(path_points))
sim.close()

# Dataset tools

In [1]:
# generate a split with only single episode
import gzip
from pathlib import Path
import json
import os
ep_id = 120
split = "val_seen"
data_folder = Path("/share/home/tj90055/hzt/IA-HWP/data/datasets/R2R_VLNCE_v1-2_preprocessed")
unseen_folder = data_folder / split
single_folder = data_folder / "single{}".format(ep_id)
single_name = "single{}".format(ep_id)
os.system("cp -r {} {}".format(str(unseen_folder), str(single_folder)))
os.makedirs(single_folder, exist_ok=True)
with gzip.open(data_folder / split / f"{split}.json.gz", "r") as f:
    single_data = json.load(f)
single_episode = []
for v in single_data["episodes"]:
    if v["episode_id"] == ep_id:
        single_episode.append(v)
single_data["episodes"] = single_episode
with gzip.open(data_folder / single_name / (single_name+".json.gz"), "w") as f:
    f.write(json.dumps(single_data).encode("utf-8"))
unseen_gt = data_folder/split/f"{split}_gt.json.gz"
single_gt = data_folder/f"single{ep_id}"/f"single{ep_id}_gt.json.gz"
os.system("cp {} {}".format(str(unseen_gt), str(single_gt)))

0

In [3]:
from pathlib import Path
best_sr = 0
best_spl = 0
best_sr_file = ""
best_spl_file = ""
folder = Path("/share/home/tj90055/hzt/IA-HWP/logs/eval_results")
for v in folder.glob("*/stats_ckpt_ckpt_*_val_unseen.json"):
    with open(v, "r") as f:
        data = json.load(f)
    if data["success"]>best_sr:
        best_sr = data["success"]
        best_sr_file = str(v)
    if data["spl"]>best_spl:
        best_spl = data["spl"]
        best_spl_file = str(v)
print(best_sr, best_sr_file)
print(best_spl, best_spl_file)

0.40348015225666123 /share/home/tj90055/hzt/IA-HWP/logs/eval_results/iaw_train10/stats_ckpt_ckpt_8_val_unseen.json
0.3654946212209901 /share/home/tj90055/hzt/IA-HWP/logs/eval_results/iaw_train10/stats_ckpt_ckpt_8_val_unseen.json


In [4]:
from pathlib import Path
import json
best_sr = 0
best_spl = 0
best_sr_file = ""
best_spl_file = ""
folder = Path("/share/home/tj90055/hzt/IA-HWP/logs/eval_results")
for v in folder.glob("*/stats_ckpt_ckpt_*_val_seen.json"):
    with open(v, "r") as f:
        data = json.load(f)
    if data["success"]>best_sr:
        best_sr = data["success"]
        best_sr_file = str(v)
    if data["spl"]>best_spl:
        best_spl = data["spl"]
        best_spl_file = str(v)
print(best_sr, best_sr_file)
print(best_spl, best_spl_file)

0.4987146529562982 /share/home/tj90055/hzt/IA-HWP/logs/eval_results/iaw_train5/stats_ckpt_ckpt_6_val_seen.json
0.4576714629915391 /share/home/tj90055/hzt/IA-HWP/logs/eval_results/iaw_train5/stats_ckpt_ckpt_6_val_seen.json
